总体假设：网络是一个由n个节点组成的无向连通图，各节点分别代表进程1，2...n，且每个进程都知道整张图。每个进程开始时都有一个来自固定集合V的输入值，放在指定的状态分量中。假设对于每个进程来说，每个开始状态只包含一个输入值。所有进程旨在通过把decision状态分量设置为V中的值，最终输出集合V中的决定值。

停止故障模型：在算法执行的任意点上，进程都有可能停止。特别地，进程有可能在发送消息的过程中停止

一致性问题的正确条件：
1. 一致性：没有两个进程会决定出不同的值
2. 有效性：如果所有的进程都是从相同的初始值$v\in V$开始的，那么$v$就是唯一可能的决定值
3. 终止性：所有无故障的进程最后都要做出决定

Byzantine故障模型：可能会从任意一个状态启动；可能发送任意消息；可能表现出任意的状态转换。

一致性问题的正确条件：
1. 一致性：没有两个无故障进程会决定出不同的值
2. 有效性：如果所有无故障进程都以相同的初始值$v\in V$开始，那么对于无故障进程来说，$v$就是唯一可能的决定值
3. 终止性：所有无故障的进程最后都要做出决定

FloodSet算法：是一个在同步消息传递系统中，用“泛洪 + 集合合并”来解决二值共识问题的确定性算法。

模型假设：同步系统，通信点对点可靠，最多$f$个崩溃进程，所有进程都知道$f$，无拜占庭错误

算法流程：算法运行$f+1$轮，每一轮发送、接收、合并集合
1. 初始状态(Round0)：$S_i := {v_i}$，每个进程只知道自己的初始值。
2. 第$r$轮$(1\leq r\leq f+1)$：向所有进程发送$S_i$，接收来自所有进程的集合$S_j$，然后合并($S_i := S_i \cup (接收到的S_j)$)
3. 决策规则(第$f+1$轮结束后)：if $S_i=={0}$ $\ \ $then$ \ \ $decide$ \ \ $0$\ \ \ $else$ \ $ decide$ \ $1

消息复杂度：$O((f+1)n^2)$

 优化版本OptFloodSet：
 1. 只传播“新出现的值”：每个进程维护$S_{i}$(当前已知的值集合)，$\Delta_{i}$(本轮新学到的值)。只有当$\Delta_{i}$非空时，才向其它进程发送
2. 不重复发送已经传播过的值：一旦系统中所有进程都知道该值就不再通信
3. 早停：如果在第$r$轮结束时，$S_{i}$在连续两轮内不再变化，则可直接进入决策

消息复杂度：$O(2n^2)$

In [1]:
class Process:
    def __init__(self, pid, initial_value):
        self.pid = pid
        self.initial_value = initial_value
        self.known_values = {initial_value}
        self.alive = True
        self.decision = None

    def crash(self):
        self.alive = False

    def __repr__(self):
        return f"P{self.pid}(S={self.known_values}, alive={self.alive})"
class FloodSetSimulator:
    def __init__(self, initial_values, f):
        """
        initial_values: dict {pid: 0 or 1}
        f: max crash failures
        """
        self.f = f
        self.processes = {
            pid: Process(pid, v)
            for pid, v in initial_values.items()
        }
        self.n = len(self.processes)
        self.round = 0

    def crash_process(self, pid):
        self.processes[pid].crash()

    def run(self, verbose=True):
        total_rounds = self.f + 1

        for r in range(1, total_rounds + 1):
            self.round = r
            if verbose:
                print(f"\n=== Round {r} ===")

            # 收集消息（同步模型）
            messages = {}

            for p in self.processes.values():
                if p.alive:
                    messages[p.pid] = p.known_values.copy()
                    if verbose:
                        print(f"P{p.pid} sends {p.known_values}")

            # 接收并合并
            for p in self.processes.values():
                if not p.alive:
                    continue
                for sender_set in messages.values():
                    p.known_values |= sender_set

            if verbose:
                for p in self.processes.values():
                    if p.alive:
                        print(f"P{p.pid} now knows {p.known_values}")

        # 决策阶段
        for p in self.processes.values():
            if not p.alive:
                continue
            if p.known_values == {0}:
                p.decision = 0
            else:
                p.decision = 1

        if verbose:
            print("\n=== Decisions ===")
            for p in self.processes.values():
                if p.alive:
                    print(f"P{p.pid} decides {p.decision}")

    def decisions(self):
        return {
            p.pid: p.decision
            for p in self.processes.values()
            if p.alive
        }
if __name__ == "__main__":
    initial_values = {
        1: 0,
        2: 1,
        3: 1,
        4: 0
    }
    f = 1
    sim = FloodSetSimulator(initial_values, f)
    # 模拟一个进程在第 1 轮前崩溃
    sim.crash_process(3)
    sim.run(verbose=True)
    print("\nFinal decisions:", sim.decisions())


=== Round 1 ===
P1 sends {0}
P2 sends {1}
P4 sends {0}
P1 now knows {0, 1}
P2 now knows {0, 1}
P4 now knows {0, 1}

=== Round 2 ===
P1 sends {0, 1}
P2 sends {0, 1}
P4 sends {0, 1}
P1 now knows {0, 1}
P2 now knows {0, 1}
P4 now knows {0, 1}

=== Decisions ===
P1 decides 1
P2 decides 1
P4 decides 1

Final decisions: {1: 1, 2: 1, 4: 1}


EIGStop算法：是一种在同步消息传递系统中，容忍崩溃故障的“早停共识算法”，用“指数信息收集（EIG 树）+ 早停判定”来保证在最坏$f$轮、最好更少轮内完成共识。

模型假设：同步系统，通信点对点可靠，最多$f$个崩溃进程，所有进程都知道$f$，无拜占庭错误

算法流程：对每个进程$p_i$

第$r$轮(1\leq r\leq f+1)：
1. 发送：进程向所有人发送：自己的EIG树中深度$\leq r-1$的所有条目
2. 接收：收到其它进程的EIG条目，按路径扩展写入自己的树
3. 剪枝L如果某条路径中出现缺失，则丢弃该分支
4. 早停检测：若满足所有深度$\leq r$的EIG的树叶节点值一致，则STOP并决定

In [2]:
class Process:
    def __init__(self, pid, initial_value):
        self.pid = pid
        self.alive = True
        self.eig = {(): initial_value}   # 空路径 = 自己的初始值
        self.decision = None

    def crash(self):
        self.alive = False
class EIGStopSimulator:
    def __init__(self, initial_values, f):
        """
        initial_values: {pid: 0 or 1}
        f: max crash failures
        """
        self.f = f
        self.processes = {
            pid: Process(pid, v)
            for pid, v in initial_values.items()
        }
        self.round = 0

    def crash(self, pid):
        self.processes[pid].crash()

    def run(self, verbose=True):
        max_rounds = self.f + 1

        for r in range(1, max_rounds + 1):
            self.round = r
            if verbose:
                print(f"\n=== Round {r} ===")

            # 发送阶段：收集每个存活进程的 EIG 条目
            messages = {}
            for p in self.processes.values():
                if p.alive:
                    messages[p.pid] = p.eig.copy()
                    if verbose:
                        print(f"P{p.pid} sends {p.eig}")

            # 接收阶段：扩展路径
            for p in self.processes.values():
                if not p.alive:
                    continue
                new_entries = {}
                for sender, eig_map in messages.items():
                    if sender == p.pid:
                        continue
                    for path, val in eig_map.items():
                        # 避免路径中重复进程
                        if sender in path or p.pid in path:
                            continue
                        new_path = (sender,) + path
                        if new_path not in p.eig:
                            new_entries[new_path] = val

                p.eig.update(new_entries)

            # ===== Early Stopping 检测 =====
            if self.can_stop(verbose):
                if verbose:
                    print("\nEarly stopping triggered!")
                break

        self.decide(verbose)

    def can_stop(self, verbose):
        """
        若所有存活进程在当前轮次看到的
        所有路径叶子值一致，则可以停止
        """
        alive = [p for p in self.processes.values() if p.alive]
        if not alive:
            return False

        reference = None
        for p in alive:
            leaf_values = set(p.eig.values())
            if len(leaf_values) > 1:
                return False
            val = next(iter(leaf_values))
            if reference is None:
                reference = val
            elif reference != val:
                return False

        if verbose:
            print("All alive processes see identical values:", reference)
        return True
    def decide(self, verbose=True):
        if verbose:
            print("\n=== Decisions ===")

        for p in self.processes.values():
            if not p.alive:
                continue
            values = set(p.eig.values())
            # 决策规则：0-only → 0，否则 → 1
            p.decision = 0 if values == {0} else 1
            if verbose:
                print(f"P{p.pid} decides {p.decision}")
if __name__ == "__main__":
    initial = {
        1: 0,
        2: 1,
        3: 1,
        4: 1
    }

    sim = EIGStopSimulator(initial, f=2)

    # 模拟一个进程崩溃
    sim.crash(3)

    sim.run(verbose=True)



=== Round 1 ===
P1 sends {(): 0}
P2 sends {(): 1}
P4 sends {(): 1}

=== Round 2 ===
P1 sends {(): 0, (2,): 1, (4,): 1}
P2 sends {(): 1, (1,): 0, (4,): 1}
P4 sends {(): 1, (1,): 0, (2,): 1}

=== Round 3 ===
P1 sends {(): 0, (2,): 1, (4,): 1, (2, 4): 1, (4, 2): 1}
P2 sends {(): 1, (1,): 0, (4,): 1, (1, 4): 1, (4, 1): 0}
P4 sends {(): 1, (1,): 0, (2,): 1, (1, 2): 1, (2, 1): 0}

=== Decisions ===
P1 decides 1
P2 decides 1
P4 decides 1


EIGByz算法：EIGByz 是在同步系统中，通过构造指数级“信息树”，在最多$f$个拜占庭故障下实现确定性共识的经典算法。

模型假设：$n\geq 3f+1$

算法流程：
1. 初始状态$Round0$：对每个进程$p_i$，$EIG_i[ < > ]:=v_i$
2. 第$r$轮($1\leq r\leq f+1$)：每个进程向所有进程发送EIG树中所有深度为$r-1$的节点；收到来自其它进程的EIG条目并对每条路径进行扩展$<j, path>$；一致性过滤，如果同一路径从不同来源收到不同值，则标记为不确定
3. 决策阶段：在第$f+1$轮结束后，每个非叶节点取其子节点的多数值，若无多数则不确定；根的值为最终决定

In [3]:
import random

class Process:
    def __init__(self, pid, initial_value, byzantine=False):
        self.pid = pid
        self.initial_value = initial_value
        self.byzantine = byzantine
        self.alive = True
        self.eig = {(): initial_value}   # EIG 树（压缩）
        self.decision = None

    def send(self, receiver_id):
        """
        拜占庭进程可以对不同接收者发送不同值
        """
        if not self.byzantine:
            return self.eig.copy()
        else:
            fake = {}
            for path in self.eig:
                fake[path] = random.choice([0, 1])
            return fake
class EIGByzSimulator:
    def __init__(self, initial_values, byzantine_set, f):
        """
        initial_values: {pid: 0 or 1}
        byzantine_set: set of Byzantine pids
        f: max Byzantine failures
        """
        self.f = f
        self.round = 0
        self.processes = {
            pid: Process(pid, v, pid in byzantine_set)
            for pid, v in initial_values.items()
        }
    def run(self, verbose=True):
        max_rounds = self.f + 1

        for r in range(1, max_rounds + 1):
            self.round = r
            if verbose:
                print(f"\n=== Round {r} ===")

            messages = {}

            # 发送阶段
            for sender in self.processes.values():
                for receiver in self.processes.values():
                    if sender.pid == receiver.pid:
                        continue
                    if not sender.alive:
                        continue
                    messages.setdefault(receiver.pid, {})[sender.pid] = \
                        sender.send(receiver.pid)

            # 接收阶段：构造新路径
            for p in self.processes.values():
                if not p.alive:
                    continue

                new_entries = {}

                for sender_id, eig_map in messages.get(p.pid, {}).items():
                    for path, val in eig_map.items():
                        if sender_id in path or p.pid in path:
                            continue
                        new_path = (sender_id,) + path
                        if len(new_path) <= max_rounds:
                            if new_path not in p.eig:
                                new_entries[new_path] = val

                p.eig.update(new_entries)

            if verbose:
                for p in self.processes.values():
                    print(f"P{p.pid} EIG:", p.eig)

        self.decide(verbose)
    def decide(self, verbose=True):
        if verbose:
            print("\n=== Decision Phase ===")

        for p in self.processes.values():
            if not p.alive:
                continue

            # 按路径长度从长到短排序
            paths = sorted(p.eig.keys(), key=len, reverse=True)
            value_map = dict(p.eig)

            # 自底向上多数
            for path in paths:
                children = [
                    value_map[c]
                    for c in value_map
                    if len(c) == len(path) + 1 and c[1:] == path
                ]
                if children:
                    zeros = children.count(0)
                    ones = children.count(1)
                    if zeros > ones:
                        value_map[path] = 0
                    elif ones > zeros:
                        value_map[path] = 1
                    # 平局保持原值（教学简化）

            p.decision = value_map[()]
            if verbose:
                print(f"P{p.pid} decides {p.decision}")

if __name__ == "__main__":
    initial_values = {
        1: 0,
        2: 0,
        3: 1,
        4: 1
    }

    byzantine = {3}   # P3 是拜占庭进程
    f = 1             # 允许 1 个拜占庭

    sim = EIGByzSimulator(initial_values, byzantine, f)
    sim.run(verbose=True)



=== Round 1 ===
P1 EIG: {(): 0, (2,): 0, (3,): 0, (4,): 1}
P2 EIG: {(): 0, (1,): 0, (3,): 0, (4,): 1}
P3 EIG: {(): 1, (1,): 0, (2,): 0, (4,): 1}
P4 EIG: {(): 1, (1,): 0, (2,): 0, (3,): 1}

=== Round 2 ===
P1 EIG: {(): 0, (2,): 0, (3,): 0, (4,): 1, (2, 3): 0, (2, 4): 1, (3, 2): 0, (3, 4): 0, (4, 2): 0, (4, 3): 1}
P2 EIG: {(): 0, (1,): 0, (3,): 0, (4,): 1, (1, 3): 0, (1, 4): 1, (3, 1): 0, (3, 4): 1, (4, 1): 0, (4, 3): 1}
P3 EIG: {(): 1, (1,): 0, (2,): 0, (4,): 1, (1, 2): 0, (1, 4): 1, (2, 1): 0, (2, 4): 1, (4, 1): 0, (4, 2): 0}
P4 EIG: {(): 1, (1,): 0, (2,): 0, (3,): 1, (1, 2): 0, (1, 3): 0, (2, 1): 0, (2, 3): 0, (3, 1): 1, (3, 2): 1}

=== Decision Phase ===
P1 decides 0
P2 decides 0
P3 decides 0
P4 decides 0


TurpinCoan算法：是在同步系统中，利用“认证消息（签名）”将拜占庭共识的轮数从$f+1$降到$2$轮的经典算法

认证通信：每条消息都带有不可伪造的签名，任意进程都可以验证：消息是谁发的，是否被篡改

算法流程：
1. Round1：广播带签名的初始值：每个$p_i$向所有进程发送$<v_i, sig_i<v_i>>$，包含初始值和自己的数字签名；接收后，本地记录$support_i[v]={所有签名支持$v$的进程}$
2. Round2：广播“支持证据”：如果看到某个值$v$获得了$\geq 2f+1$个不同签名支持，则广播$<v, {sig_{j(v)}}>$
3. 决策规则：若收到某值$v$且附带$\geq 2f+1$个合法签名，则decide $v$

In [4]:
import random

class Process:
    def __init__(self, pid, initial_value, byzantine=False):
        self.pid = pid
        self.initial_value = initial_value
        self.byzantine = byzantine
        self.decision = None

    def sign(self, value):
        """
        抽象签名：只能给自己的 value 签名
        """
        return (self.pid, value)

    def round1_message(self, receiver_id):
        """
        第一轮发送初始值
        拜占庭进程可以对不同接收者撒谎
        """
        if not self.byzantine:
            return self.sign(self.initial_value)
        else:
            fake_value = random.choice([0, 1])
            return self.sign(fake_value)
class TurpinCoanSimulator:
    def __init__(self, initial_values, byzantine_set, f):
        """
        initial_values: {pid: 0 or 1}
        byzantine_set: set of Byzantine process IDs
        f: max Byzantine failures
        """
        self.f = f
        self.processes = {
            pid: Process(pid, v, pid in byzantine_set)
            for pid, v in initial_values.items()
        }
    def round1(self, verbose=True):
        """
        Round 1: 每个进程向所有进程发送 (value, signature)
        """
        inbox = {pid: [] for pid in self.processes}

        for sender in self.processes.values():
            for receiver in self.processes.values():
                msg = sender.round1_message(receiver.pid)
                inbox[receiver.pid].append(msg)

        if verbose:
            print("\n=== Round 1 Messages ===")
            for pid, msgs in inbox.items():
                print(f"P{pid} receives:", msgs)

        return inbox
    def round2(self, inbox1, verbose=True):
        """
        Round 2: 广播达到 2f+1 签名的值
        """
        inbox2 = {pid: [] for pid in self.processes}

        for pid, msgs in inbox1.items():
            support = {0: set(), 1: set()}

            for signer, value in msgs:
                support[value].add(signer)

            for value, signers in support.items():
                if len(signers) >= 2 * self.f + 1:
                    # 构造证据包
                    certificate = (value, frozenset(signers))
                    for receiver in inbox2:
                        inbox2[receiver].append(certificate)

        if verbose:
            print("\n=== Round 2 Certificates ===")
            for pid, certs in inbox2.items():
                print(f"P{pid} receives:", certs)

        return inbox2
    def decide(self, inbox2, verbose=True):
        if verbose:
            print("\n=== Decisions ===")

        for pid, certs in inbox2.items():
            for value, signers in certs:
                if len(signers) >= 2 * self.f + 1:
                    self.processes[pid].decision = value
                    break

            if self.processes[pid].decision is None:
                self.processes[pid].decision = 0  # 默认值（理论中可定义）

            if verbose:
                print(f"P{pid} decides {self.processes[pid].decision}")
    def run(self, verbose=True):
        inbox1 = self.round1(verbose)
        inbox2 = self.round2(inbox1, verbose)
        self.decide(inbox2, verbose)
if __name__ == "__main__":
    initial_values = {
        1: 0,
        2: 0,
        3: 1,
        4: 0
    }

    byzantine = {3}   # P3 是拜占庭进程
    f = 1             # 允许 1 个拜占庭

    sim = TurpinCoanSimulator(initial_values, byzantine, f)
    sim.run(verbose=True)



=== Round 1 Messages ===
P1 receives: [(1, 0), (2, 0), (3, 1), (4, 0)]
P2 receives: [(1, 0), (2, 0), (3, 0), (4, 0)]
P3 receives: [(1, 0), (2, 0), (3, 0), (4, 0)]
P4 receives: [(1, 0), (2, 0), (3, 1), (4, 0)]

=== Round 2 Certificates ===
P1 receives: [(0, frozenset({1, 2, 4})), (0, frozenset({1, 2, 3, 4})), (0, frozenset({1, 2, 3, 4})), (0, frozenset({1, 2, 4}))]
P2 receives: [(0, frozenset({1, 2, 4})), (0, frozenset({1, 2, 3, 4})), (0, frozenset({1, 2, 3, 4})), (0, frozenset({1, 2, 4}))]
P3 receives: [(0, frozenset({1, 2, 4})), (0, frozenset({1, 2, 3, 4})), (0, frozenset({1, 2, 3, 4})), (0, frozenset({1, 2, 4}))]
P4 receives: [(0, frozenset({1, 2, 4})), (0, frozenset({1, 2, 3, 4})), (0, frozenset({1, 2, 3, 4})), (0, frozenset({1, 2, 4}))]

=== Decisions ===
P1 decides 0
P2 decides 0
P3 decides 0
P4 decides 0


一致广播算法：

模型假设：
1. 同步系统：消息在固定时间内送达
2. 进程失败：Crash 或 Byzantine
3. 通道：可靠 / FIFO 通道，或可被拜占庭污染
4. 目标：即使最多$f$个进程失败，也保证正确进程达成一致

核心思想：
1. 泛洪消息：广播者发送消息给所有进程；每个进程收到后再转发给其他进程
2. 多数确认 / 多轮确认：只有收到足够多的转发确认后，才“交付”消息；防止拜占庭或丢包导致分歧

PolyByz算法：

系统模型：
1. n 个进程，其中最多$f$个拜占庭
2. 同步消息传递系统(每轮消息按轮次到达)
3. 每个进程有初始值$v_i$(可多值，不是二值)

核心思想：
1. 把值编码成多项式：每个进程把自己初始值视为 常数项的多项式；通过选择若干随机点，把信息嵌入到多项式系数中
2. 消息传播：点的泛洪：每个进程将其多项式在不同点上的值发送给其他进程；拜占庭进程可能发送错误的点值
3. 收集信息后进行多项式插值：正确进程收集 ≥ n-f 个点；拜占庭进程最多破坏 f 个点；利用多项式插值恢复正确的多项式
4. 最终决策：恢复出的多项式的常数项作为最终决策值；保证一致性与合法性

算法流程：
1. Round 1：每个进程广播自己的多项式的点值
2. Round 2：进程转发收到的点值（帮助拜占庭点值被过滤）
3. Round f+1（理论上最多 f+1 轮）：插值恢复多项式并决策

核心优势是通信量从$O(n^{f+1})$降到$O(n^2 f)$

In [5]:
import random

class Process:
    def __init__(self, pid, initial_value, f, byzantine=False):
        self.pid = pid
        self.initial_value = initial_value  # 进程初始值
        self.f = f                          # 最大拜占庭进程数量
        self.byzantine = byzantine
        self.points = {}                    # 收集的点: {sender_id: value}
        self.decided_value = None

    def generate_points(self, n):
        """
        把初始值编码成 n 个“点”发送给其他进程
        教学模拟：直接生成一个点（pid, value)
        """
        if self.byzantine:
            # 拜占庭进程可能乱发送不同值
            return {i: random.choice([0,1]) for i in range(1,n+1)}
        else:
            # 正确进程：广播自己的值
            return {i: self.initial_value for i in range(1,n+1)}

class PolyByzSimulator:
    def __init__(self, initial_values, byzantine_set, f):
        """
        initial_values: {pid: value}
        byzantine_set: set of pid
        f: 最大拜占庭进程数
        """
        self.n = len(initial_values)
        self.f = f
        self.processes = {
            pid: Process(pid, val, f, pid in byzantine_set)
            for pid, val in initial_values.items()
        }

    def run(self, verbose=True):
        # Round 1: 每个进程生成点发送给所有进程
        inbox = {pid: {} for pid in self.processes}  # pid -> {sender: value}

        for sender in self.processes.values():
            points = sender.generate_points(self.n)
            for receiver_pid in inbox:
                inbox[receiver_pid][sender.pid] = points[receiver_pid]

        if verbose:
            print("\n=== Round 1: Points Received ===")
            for pid, points in inbox.items():
                print(f"P{pid} received: {points}")

        # Round 2: 转发收到的点（模拟多轮传播）
        # 教学模拟：直接收集所有点即可
        final_points = {pid: {} for pid in self.processes}

        for pid in self.processes:
            # 收集所有进程的点
            for sender_pid in inbox:
                final_points[pid][sender_pid] = inbox[sender_pid][pid]

        if verbose:
            print("\n=== Round 2: Collected Points ===")
            for pid, pts in final_points.items():
                print(f"P{pid} points: {pts}")

        # 决策：选出出现次数最多的值
        for pid, pts in final_points.items():
            counts = {0:0, 1:0}
            for v in pts.values():
                counts[v] += 1
            # 拜占庭最多 f 个 -> 选择出现次数 >= n-f 的值
            for val, cnt in counts.items():
                if cnt >= self.n - self.f:
                    self.processes[pid].decided_value = val
                    break
            if self.processes[pid].decided_value is None:
                self.processes[pid].decided_value = 0  # 默认值

        if verbose:
            print("\n=== Decisions ===")
            for pid, proc in self.processes.items():
                print(f"P{pid} decides {proc.decided_value}")

    def get_decisions(self):
        return {pid: proc.decided_value for pid, proc in self.processes.items()}


if __name__ == "__main__":
    # 测试场景
    initial_values = {1:0, 2:0, 3:1, 4:0}  # 初始值
    byzantine_set = {3}                    # P3 是拜占庭
    f = 1                                  # 最大拜占庭数

    sim = PolyByzSimulator(initial_values, byzantine_set, f)
    sim.run(verbose=True)



=== Round 1: Points Received ===
P1 received: {1: 0, 2: 0, 3: 1, 4: 0}
P2 received: {1: 0, 2: 0, 3: 1, 4: 0}
P3 received: {1: 0, 2: 0, 3: 1, 4: 0}
P4 received: {1: 0, 2: 0, 3: 0, 4: 0}

=== Round 2: Collected Points ===
P1 points: {1: 0, 2: 0, 3: 0, 4: 0}
P2 points: {1: 0, 2: 0, 3: 0, 4: 0}
P3 points: {1: 1, 2: 1, 3: 1, 4: 0}
P4 points: {1: 0, 2: 0, 3: 0, 4: 0}

=== Decisions ===
P1 decides 0
P2 decides 0
P3 decides 1
P4 decides 0


定理：Byzantine一致性问题可以在一个$n$节点的网络图$G$中解决，图中允许有$f$个故障进程，当且仅当一下的两个条件都得到满足：
1. $n> 3f$
2. $conn(G)> 2f$

conn(G)表示最小的节点数，这种节点满足，如果移去这些节点，图形就变成非连通图或者平凡的只有一个节点的图。